In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import confusion_matrix , classification_report 
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_dir = "../input/emotion-detection-fer/train"
test_dir = "../input/emotion-detection-fer/test"

image_size = 48

In [ ]:
train_datagen = ImageDataGenerator(rotation_range = 180,
                                         width_shift_range = 0.1,
                                         height_shift_range = 0.1,
                                         horizontal_flip = True,
                                         rescale = 1./255,
                                         zoom_range = 0.2,
                                         validation_split = 0.2
                                        )
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)

train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (image_size,image_size),
                                                    batch_size = 64,
                                                    shuffle=True,
                                                    color_mode = "rgb",
                                                    seed=12,
                                                    class_mode = "categorical",
                                                    subset = "training"
                                                   )
validation_generator = validation_datagen.flow_from_directory(directory = train_dir,
                                                              target_size = (image_size,image_size),
                                                              batch_size = 64,
                                                              shuffle=True,
                                                              color_mode = "rgb",
                                                              class_mode = "categorical",
                                                              subset = "validation",
                                                              seed = 12
                                                             )
test_generator = validation_datagen.flow_from_directory(directory= test_dir,
                                                        target_size= (image_size, image_size),
                                                        batch_size= 64,
                                                        shuffle=False,
                                                        color_mode = 'rgb',
                                                        class_mode='categorical',
                                                        seed = 12 
)

In [ ]:
base_model = tf.keras.applications.ResNet50(input_shape = (image_size, image_size, 3), include_top = False, weights = 'imagenet')
inputs = tf.keras.Input(shape = (image_size, image_size,3))
out = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(out)
x = tf.keras.layers.Dense(256, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(1024, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.Dropout(0.5) (x)
x = tf.keras.layers.Dense(7, activation="softmax", name="classification")(x)

model = tf.keras.Model(inputs = inputs, outputs = x)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.layers[1].trainable = False

In [ ]:
model.summary()

In [ ]:
history = model.fit_generator(train_generator,
                    epochs = 50 ,
                    validation_data = validation_generator
                    )

history = pd.DataFrame(history.history)

In [ ]:
# Un-Freezing the feature extraction layers for fine tuning 
model.layers[1].trainable = True

model.compile(optimizer=tf.keras.optimizers.SGD(0.001), #lower learning rate
                loss='categorical_crossentropy',
                metrics = ['accuracy'])

history_ = model.fit_generator(train_generator,epochs = 50 ,validation_data = validation_generator)
history = history.append(pd.DataFrame(history_.history) , ignore_index=True)

In [ ]:
model.save("model_emotion.h5")

In [ ]:
model.evaluate(test_generator)
preds = model.predict(test_generator)
y_preds = np.argmax(preds , axis = 1 )
y_test = np.array(test_generator.labels)

In [ ]:
x = px.line(data_frame= history , y= ["accuracy" , "val_accuracy"] ,markers = True )
x.update_xaxes(title="Number of Epochs")
x.update_yaxes(title = "Accuracy")
x.update_layout(showlegend = True,
    title = {
        'text': 'Accuracy vs Number of Epochs',
        'y':0.94,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
x.show()

In [ ]:
x = px.line(data_frame= history , 
            y= ["loss" , "val_loss"] , markers = True )
x.update_xaxes(title="Number of Epochs")
x.update_yaxes(title = "Loss")
x.update_layout(showlegend = True,
    title = {
        'text': 'Loss vs Number of Epochs',
        'y':0.94,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
x.show()

In [ ]:
model.save('model_emotion.h5')